<a href="https://colab.research.google.com/github/DmitrijMakarov/cosmo_luch/blob/main/luch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Импорт необходимых библиотек###

In [1]:
!pip3 install catboost
import catboost as cat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
import keras
from tensorflow import keras
from keras import layers

ModuleNotFoundError: No module named 'tensorflow'

###Считывание данных###

In [2]:
def read_binarry(file_path):
    import struct
    import math

    power = []
    age = []
    coordinate_x = []
    coordinate_y = []
    angle_tetta = []
    angle_phi = []
    energy = []
    time = []

    with open(file_path, 'rb') as binary_file:
        for i in range(100000):
            binary_file.read(4 * 5)

            tetta = struct.unpack('f', binary_file.read(4))[0]
            angle_tetta.append(tetta)

            phi = struct.unpack('f', binary_file.read(4))[0]
            angle_phi.append(phi)

            x0 = struct.unpack('f', binary_file.read(4))[0]
            coordinate_x.append(x0)

            y0 = struct.unpack('f', binary_file.read(4))[0]
            coordinate_y.append(y0)

            binary_file.read(4 * 5)

            power_eas = struct.unpack('f', binary_file.read(4))[0]
            power.append(math.log10(power_eas))

            age_eas = struct.unpack('f', binary_file.read(4))[0]
            age.append(age_eas)

            binary_file.read(4 * 1565)
            energy_release = struct.unpack('f' * 36, binary_file.read(4 * 36))
            energy.append(energy_release)

            binary_file.read(4)
            t = struct.unpack('f' * 144, binary_file.read(4 * 144))
            threshold_time = t[::4]
            time.append(threshold_time)


    # Собираем всё в DataFrame
    df = pd.DataFrame({
        'power': power,
        'age': age,
        'x': coordinate_x,
        'y': coordinate_y,
        'tetta': angle_tetta,
        'phi': angle_phi,
        'energy': energy,
        'threshold_time': time,
    })

    y_data = pd.DataFrame({
        'power': power,
        'age': age,
        'x': coordinate_x,
        'y': coordinate_y,
        'tetta': angle_tetta,
        'phi': angle_phi,})

    return df, y_data

df, y_data = read_binarry('spe27p_100k_2022_correct.dat')

In [3]:
df

,power,age,x,y,tetta,phi,energy,threshold_time
0,4.166507,1.444716,13.572407,37.022316,37.367474,47.313324,"(0.10191129148006439, 11.324358940124512, 13.0...","(-1.0, 91830.7265625, 91844.828125, 91830.7109..."
1,4.933104,1.430547,40.330677,-61.980999,28.912228,236.400421,"(10.563158988952637, 0.0, 7.604449272155762, 1...","(61243.13671875, -1.0, 61215.59375, 61216.8359..."
2,4.133581,1.459046,-1.046695,-63.925430,37.267426,182.086060,"(0.0, 0.34477588534355164, 0.35262614488601685...","(-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 78681.062..."
3,5.190620,1.326983,40.293152,-21.836197,3.205503,201.268890,"(34.960296630859375, 40.36423873901367, 6.7627...","(52147.2421875, 52211.25, 52142.87890625, 5213..."
4,5.021614,1.336776,15.542248,10.037846,6.299241,108.097626,"(79.6240234375, 29.508312225341797, 60.4609146...","(88044.953125, 88053.078125, 88046.9453125, 88..."
...,...,...,...,...,...,...,...,...
99995,4.503942,1.295468,36.338993,32.139332,44.537529,31.058533,"(8.940515518188477, 0.0, 0.0, 1.29427850246429...","(96596.9765625, -1.0, -1.0, -1.0, 96563.203125..."
99996,5.301322,1.302831,-29.983866,-38.277843,16.121550,158.978851,"(103.17373657226562, 123.92174530029297, 360.8...","(74179.7890625, 74196.1640625, 74172.7578125, ..."
99997,3.617597,1.473757,18.329506,-13.626085,46.820038,13.862854,"(0.0, 0.23389649391174316, 0.0, 1.705774307250...","(-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
99998,4.017301,1.510002,-36.200687,-53.555149,33.087467,157.593292,"(0.0, 0.0, 8.356832504272461, 16.7534923553466...","(-1.0, -1.0, 166325.0, 166352.15625, -1.0, -1...."


In [4]:
X_data = pd.DataFrame({})

for i in range(36):
    _energy = []
    for j in range(len(df["energy"])):
        _energy.append(df['energy'][j][i])
    X_data[f'energy{i}'] = _energy
for i in range(36):
    _time = []
    for j in range(len(df["threshold_time"])):
        _time.append(df['threshold_time'][j][i])
    X_data[f'time{i}'] = _time

In [5]:
X = X_data.copy()

###Предсказывание значений оси x###

In [6]:
y_x = df['x']
X_train_x, X_test_x, y_train_x, y_test_x = train_test_split(X, y_x, test_size=0.2)

In [7]:
gb_x = GradientBoostingRegressor()
gb_x.fit(X_train_x, y_train_x)

GradientBoostingRegressor()

In [8]:
y_pred_x = gb_x.predict(X_test_x)

In [10]:
r2_score(y_pred_x, y_test_x)

0.6879429957984307

In [11]:
y_pred_x

array([-26.08502558,  24.54116608, -30.78164791, ...,  17.15604323,
        24.66096627,  17.00394914])

###Предсказывание значений оси y###

In [12]:
y_y = df['y']
X_train_y, X_test_y, y_train_y, y_test_y = train_test_split(X, y_y, test_size=0.2)

In [13]:
gb_y = GradientBoostingRegressor()
gb_y.fit(X_train_y, y_train_y)

GradientBoostingRegressor()

In [14]:
y_pred_y = gb_y.predict(X_test_y)

In [15]:
r2_score(y_pred_y, y_test_y)

0.855153791763329

In [16]:
y_pred_y

array([ 25.96910261,  28.15863336, -20.25902825, ...,  -9.27775914,
        -7.43729352,  32.9693386 ])

###Предсказание углов###

In [19]:
y_angles = df[['tetta', 'phi']]

In [20]:
y_angles

,tetta,phi
0,37.367474,47.313324
1,28.912228,236.400421
2,37.267426,182.086060
3,3.205503,201.268890
4,6.299241,108.097626
...,...,...
99995,44.537529,31.058533
99996,16.121550,158.978851
99997,46.820038,13.862854
99998,33.087467,157.593292


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, y_angles, random_state=1234, test_size=0.2)
# angles
model = keras.Sequential(
    [
        layers.Dense(72, activation='relu'),
        layers.Dense(300, activation='relu'),
        layers.Dense(300, activation='relu'),
        layers.Dense(300, activation='relu'),
        layers.Dense(2, activation='softplus'),
    ]
)

model.compile(
    optimizer='AdamW',
    loss='categorical_crossentropy',
    metrics=["accuracy"],
    )

history = model.fit(
    X_train,
    Y_train,
    batch_size=128,
    epochs=5,
    validation_split=0.1
)